In [1]:
from model_phantom_DB import *
from linares_plot import * 
import itertools

%matplotlib inline
%config InlineBackend.figure_format = 'png' ## 'svg'

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
import pandas as pd

For all the matrice data : each row is one trial, each column is one neuron. 
For the same row the trial data may not be collected on the same trial since data might are collected on different days.

cuerate: firing rate during 1st visual stimuli(cue) presentation.

cuerate_stim: firing rate during 1st cue presentation with NB stimulation. 

cdrate: firing rate during the 1st delay after 1st cue presentation. 

cdrate_stim: firing rate during the 1st delay after cue presentation  with NB stimulation.  
 
samplerate: firing rate during 2nd visual stimuli(sample) presentation. 

samplerate_stim: firing rate during sample presentation with NB stimulation. 

sdrate: firing rate during the delay after sample presentation. 

sdrate_stim: firing rate during the delay after sample presentation with NB stimulation. 

g1: location of 1st visual stimuli(cue)
 value -1 means no visual stimuli presented.
 value [1 2 3 4 5] are  relative locaions, such as [0 45 90 135 180], or [ 45 90 135 180 225], etc.
 The real locations depended on neurons' receptive field.
 According to our behavioral paradigm, the cue will only be presented on locaion [1 5]. that is two opposite locations, such as [0 180], [45 225], etc.
 For each neuron, thoes value([1:5]) refer to the same locations.

g2: location of 2nd visual stimuli(sample)
  Sample can show up in all five possible locations([1:5]) or not presented(-1).

g3: task type for each trial, eg. saccade to 1st (value 1), or saccade to 2nd(value 2).


In [4]:
from scipy.io import loadmat 
file = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Classifiy_data_odrdistvar_allNeurons_reverse.matlab'
data = loadmat(file)
data.keys()

dict_keys(['cdrate_stim', 'g3_stim', 'samplerate', 'cuerate_stim', 'g1_stim', 'sdrate', 'g2', 'sdrate_stim', 'g1', 'samplerate_stim', '__version__', 'g3', 'cdrate', '__globals__', 'cuerate', '__header__', 'g2_stim'])

In [13]:
a

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,229,230,231,232
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [32]:
data['cdrate'][-5,:]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [19]:
bool_ = ~np.isnan(data['cdrate'])

In [22]:
pd.DataFrame(data['cdrate'][bool_])

,0
0,34.0
1,16.0
2,52.0
3,1.0
4,0.0
5,11.0
6,0.0
7,2.0
8,0.0
9,5.0


In [12]:
df = pd.DataFrame(data['cdrate'])
a=~df.isna()
df[a]

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,229,230,231,232
0,34.0,16.0,52.0,1.0,0.0,11.0,0.0,2.0,0.0,5.0,...,1.0,3.0,4.0,0.0,0.0,0.0,6.0,0.0,2.0,1.0
1,25.0,29.0,48.0,15.0,0.0,18.0,1.0,7.0,3.0,1.0,...,0.0,2.0,8.0,0.0,8.0,0.0,4.0,0.0,4.0,1.0
2,3.0,17.0,35.0,4.0,0.0,3.0,0.0,0.0,0.0,1.0,...,2.0,3.0,4.0,3.0,3.0,0.0,1.0,0.0,5.0,2.0
3,25.0,14.0,42.0,8.0,2.0,15.0,0.0,6.0,0.0,3.0,...,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0
4,24.0,21.0,30.0,11.0,3.0,9.0,0.0,2.0,0.0,2.0,...,4.0,8.0,5.0,0.0,2.0,0.0,3.0,2.0,9.0,1.0
5,1.0,23.0,46.0,3.0,0.0,11.0,0.0,1.0,0.0,2.0,...,2.0,4.0,2.0,4.0,0.0,0.0,1.0,0.0,4.0,1.0
6,13.0,16.0,34.0,2.0,0.0,5.0,0.0,1.0,1.0,4.0,...,4.0,5.0,6.0,0.0,4.0,0.0,3.0,0.0,4.0,0.0
7,18.0,6.0,35.0,2.0,0.0,9.0,0.0,2.0,1.0,7.0,...,0.0,2.0,4.0,0.0,0.0,0.0,3.0,1.0,5.0,3.0
8,26.0,10.0,27.0,2.0,0.0,6.0,0.0,0.0,0.0,5.0,...,3.0,2.0,3.0,4.0,1.0,0.0,6.0,0.0,5.0,0.0
9,16.0,25.0,45.0,4.0,3.0,15.0,0.0,2.0,0.0,2.0,...,1.0,9.0,5.0,0.0,5.0,0.0,6.0,0.0,7.0,0.0
